# 1. Traffic sign & light classification
Traffic signs included in contest:
- Stop
- Parking
- Crosswalk
- Priority
- Highway Entry
- Highway End
- One-way
- Roundabout
- No-entry

(European standard)

### SETUP

In [ ]:
import torch.cuda
import yaml
import logging
from pathlib import Path
from ultralytics import YOLO


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
model = YOLO("yolo11n.pt")


class YOLO11Trainer:
    def __init__(self, data_yaml, model_size='n', device='cuda'):
        self.data_yaml = data_yaml
        self.model_size = model_size
        self.device = device if torch.cuda.is_available() else 'cpu'
        
        # Load data config
        with open(data_yaml, 'r') as f:
            self.data_config = yaml.safe_load(f)
        
        self.num_classes = self.data_config['nc']

        print()
        
    def get_optimal_batch_size(self, imgsz=640):
        if self.device == 'cpu':
            return 8
        
        # GPU memory-based estimation
        gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9  # GB
        
        if gpu_mem < 6:
            return 16
        elif gpu_mem < 12:
            return 32
        else:
            return 64
    
    def train_optimized(self, epochs=200, imgsz=640):
        logger.info(f"Starting optimized training on {self.device}")
        logger.info(f"Model: YOLOv11{self.model_size}")
        logger.info(f"Classes: {self.num_classes}")
        
        # Initialize model
        model = YOLO(f'yolo11{self.model_size}.pt')
        
        # Determine optimal batch size
        batch_size = self.get_optimal_batch_size(imgsz)
        logger.info(f"Using batch size: {batch_size}")
        
        # Training configuration
        train_args = {
            # Dataset
            'data': self.data_yaml,
            
            # Training params
            'epochs': epochs,
            'patience': max(30, epochs // 10),
            'batch': batch_size,
            'imgsz': imgsz,
            
            # Hardware
            'device': self.device,
            'workers': 8,
            
            # Optimizer
            'optimizer': 'AdamW',
            'lr0': 0.001,
            'lrf': 0.01,
            'momentum': 0.937,
            'weight_decay': 0.0005,
            'warmup_epochs': 3.0,
            'warmup_momentum': 0.8,
            'warmup_bias_lr': 0.1,
            
            # Loss weights (optimized for small objects)
            'box': 7.5,
            'cls': 0.5,
            'dfl': 1.5,
            
            # Data augmentation (optimized for traffic scenarios)
            'hsv_h': 0.015,
            'hsv_s': 0.7,
            'hsv_v': 0.4,
            'degrees': 10.0,
            'translate': 0.1,
            'scale': 0.9,
            'shear': 2.0,
            'perspective': 0.0001,
            'flipud': 0.0,
            'fliplr': 0.5,
            'mosaic': 1.0,
            'mixup': 0.15,
            'copy_paste': 0.3,
            'auto_augment': 'randaugment',
            'erasing': 0.4,
            'crop_fraction': 1.0,
            'close_mosaic': 15,
            
            # Multi-scale
            'rect': False,
            
            # Validation & saving
            'val': True,
            'save': True,
            'save_period': 10,
            'plots': True,
            
            # Project
            'project': 'bosch_challenge',
            'name': f'optimized_yolo11{self.model_size}_{imgsz}',
            'exist_ok': False,
            'verbose': True,
        }
        
        # Train
        results = model.train(**train_args)
        
        logger.info("Training completed!")
        logger.info(f"Best model: {results.save_dir}/weights/best.pt")
        
        return results
    
    def train_progressive(self):
        # Warmup
        print('\n-----Warmup phase-----')
        model = YOLO(f'yolo11{self.model_size}.pt')
        results_stage1 = model.train(
            data=self.data_yaml,
            epochs=10,
            batch=self.get_optimal_batch_size(),
            freeze=10,
            lr0=0.01,
            optimizer='AdamW',
            project='BFMC_TrafficSign',
            name=f'stage1_warmup_{self.model_size}',
            imgsz=640,
            device=self.device,
        )
        
        # Full training
        print('\n-----Fulltrain phase-----')
        
        best_warmup = Path(results_stage1.save_dir) / 'weights' / 'last.pt'
        model = YOLO(str(best_warmup))
        results_stage2 = model.train(
            data=self.data_yaml,
            epochs=150,
            batch=self.get_optimal_batch_size(),
            lr0=0.001,
            optimizer='AdamW',
            mosaic=0.8,
            mixup=0.15,
            project='BFMC_TrafficSign',
            name=f'stage2_full_{self.model_size}',
            imgsz=640,
            device=self.device,
            patience=30,
        )
        
        # Fine-tuning
        print('\n-----Finetune phase-----')
        
        best_full = Path(results_stage2.save_dir) / 'weights' / 'best.pt'
        model = YOLO(str(best_full))
        results_stage3 = model.train(
            data=self.data_yaml,
            epochs=50,
            batch=self.get_optimal_batch_size() // 2,
            lr0=0.0001,
            optimizer='AdamW',
            mosaic=0.0,
            mixup=0.0,
            project='BFMC_TrafficSign',
            name=f'stage3_finetune_{self.model_size}',
            imgsz=640,
            device=self.device,
            patience=20,
        )
        
        logger.info("\n" + "="*60)
        logger.info("Progressive Training Complete!")
        logger.info(f"Final model: {results_stage3.save_dir}/weights/best.pt")
        logger.info("="*60)
        
        return results_stage3

### TRAINING

In [ ]:
res = model.train(
    data="FullIJCNN2013.yaml",
    epochs=100,
    batch=24,
    device=-1,

    classes=[1, 2, 12, 13, 14, 17, 27, 35, 38, 40],
    patience=10,
    optimizer='AdamW',
    multi_scale=True,
    lr0=1e-3,
)

### VALIDATING

In [ ]:
from glob import glob
from PIL import Image
from IPython.display import display

for path in glob('runs\\detect\\train4\\*.jpg') + glob('runs\\detect\\train4\\*.png'):
    display(Image.open(path))

### TESTING

### EXPORT MODEL

In [ ]:
model = YOLO('runs/detect/train/weights/best.pt')

model.export(
    format='onnx',
    imgsz=416,
    dynamic=False,
    simplify=True
)